This function creates data generators for training and validation images with rescaling.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

def create_generators(train_dir, validation_dir):
    train_datagen = ImageDataGenerator(rescale=1./255)
    validation_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
    return train_generator, validation_generator

Load the pre-trained ResNet50 model to use as the base for the new model.

In [ ]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Build a new sequential model adding the base model and additional layers.

In [ ]:
my_new_model = Sequential()
my_new_model.add(base_model)
my_new_model.add(Dense(256, activation='relu'))
my_new_model.add(Dense(num_classes, activation='softmax'))

Compile the model using the Adam optimizer, categorical crossentropy loss, and accuracy metric.

In [ ]:
my_new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Train the model using the training generator and validate with the validation generator.

In [ ]:
fit_stats = my_new_model.fit(train_generator, epochs=10, validation_data=validation_generator)

Output the fit statistics after training to evaluate the model performance.

In [ ]:
print(fit_stats.history)

Store the results in a pickle file for later analysis.

In [ ]:
import pickle

with open('fit_stats.pkl', 'wb') as f:
    pickle.dump(fit_stats.history, f)